# 📦 Mercado Libre Data Engineer Challenge
Este notebook carga datos simulados y ejecuta consultas SQL sobre una base de datos SQLite.

In [1]:
# librerias
import pandas as pd
import json
import requests
from google.colab import drive
import plotly.express as px

In [3]:
# REFRESH TOKEN para poder volver a realizar gets
import requests

url = "https://api.mercadolibre.com/oauth/token"

# Copiar el último refresh_token del output de esta celda y remplazarlo por el siguiente
payload = 'grant_type=refresh_token&client_id=6511763805540892&client_secret=wFdNuZNj1lgis0qYCCfFPKlVLXhiyzON&refresh_token=TG-68342bc8b623c60001af4a8f-232792475'
headers = {
  'accept': 'application/json',
  'content-type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"access_token":"APP_USR-6511763805540892-052604-2b2bb20e8b94fb0900312586060afd01-232792475","token_type":"Bearer","expires_in":21600,"scope":"offline_access read","user_id":232792475,"refresh_token":"TG-68342d49ca2d9b0001c7fc3c-232792475"}


In [34]:
# Utilizo el recurso de Predictor de categorías para obtener la categoría de productos buscados
# Lista de términos de búsqueda
busquedas = ['celular iphone', 'cafetera', 'televisor', 'notebook', 'calefactor', 'Colchonetas', 'campera montagne', 'monitor%20pc']
site = 'MLA'

# Token de acceso (si es necesario para autenticación)
headers = {
    'Authorization': f'Bearer {access_token}'  # solo si tu token es requerido
}

# Lista para almacenar resultados
resultados = []

# Iterar sobre los términos
for termino in busquedas:
    query = requests.utils.quote(termino)
    url = f"https://api.mercadolibre.com/sites/{site}/domain_discovery/search?q={query}"

    response = requests.get(url, headers=headers)
    data = response.json()
    category_id = data[0]["category_id"]
    category_name = data[0]["category_name"]
    resultados.append({
        "busqueda": termino,
        "category_id": category_id,
        "category_name": category_name
    })
    print(f"✅ {termino} → {category_id} ({category_name})")

# Convertir a DataFrame y guardar en CSV
df_resultados = pd.DataFrame(resultados)

✅ celular iphone → MLA1055 (Celulares y Smartphones)
✅ cafetera → MLA4340 (Cafeteras)
✅ televisor → MLA381962 (Televisores)
✅ notebook → MLA1652 (Notebooks)
✅ calefactor → MLA438143 (A Gas)
✅ Colchonetas → MLA44765 (Mats y Colchonetas)
✅ campera montagne → MLA109096 (Camperas, Tapados y Trenchs)
✅ monitor%20pc → MLA14407 (Monitores)


In [43]:
category_ids = [item['category_id'] for item in resultados]

In [44]:
# Utilizo el recurso de Más vendidos por categoría para obtener el id de productos
# Lista de category_id (obtenidos previamente)

# Token de acceso (si es necesario para autenticación)
headers = {
    'Authorization': f'Bearer {access_token}'  # solo si tu token es requerido
}

# Almacenar los productos destacados por categoría
destacados = []

for categoria in category_ids:
    url = f"https://api.mercadolibre.com/highlights/{site}/category/{categoria}"
    response = requests.get(url, headers=headers)
    data = response.json()
    productos = data.get("content", [])
    for producto in productos:
        destacados.append({
            "category_id": categoria,
            "item_id": producto.get("id"),
            "position": producto.get("position")
        })

# Convertir a DataFrame y guardar
df_destacados = pd.DataFrame(destacados)

In [45]:
df_destacados

,category_id,item_id,position
0,MLA1055,MLA40008188,1
1,MLA1055,MLA41998926,2
2,MLA1055,MLA41262061,3
3,MLA1055,MLA46272309,4
4,MLA1055,MLA42455810,5
...,...,...,...
135,MLA14407,MLA43956632,16
136,MLA14407,MLA47609708,17
137,MLA14407,MLA48937106,18
138,MLA14407,MLA36946604,19
